In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

In [2]:
## --- 설정 변수 정의 ---
## 이미지 경로 설정
# 입력 이미지 경로 (directory) 
img_path_list = "/src/data/DeepCrack/train/all_img"
# 저장 이미지 경로
save_path = "/src/hyebin/preprocessing/division_all_img/"

## 스텝 사이즈, 잘라낼 크기 설정
# 가로축 이동 거리
step_x = 20
# 세로축 이동 거리
step_y = 20
# 가로 범위
width = 50
# 세로 범위
height = 50

In [3]:
## --- 이미지 분할 및 저장 함수 ---
def division_image(img_file_name, pil_img, step_x, step_y, width, height):
    
    (img_w, img_h) = pil_img.size
    #print('img_size : {}'.format(pil_img.size))
 
    range_w = round(img_w/step_x)
    range_h = round(img_h/step_y)
    #print('range_w : {},  range_h : {}'.format(range_w, range_h))
    
    # 이미지 이름별로 폴더 만들고 저장하기
    img_file_name = img_file_name.replace(".jpg", "").replace(".png", "")
    if not os.path.exists(save_path + img_file_name):  os.mkdir(save_path + img_file_name)
    
    i = 0
    division_list = []
    fname_list = []
    for h in range(range_h):  # height
        for w in range(range_w):  # weight
            # 가로 시작점(좌), 세로 시작점(상), 가로 범위(우), 세로 범위(하)
            left = w*step_x
            top = h*step_y
            right = (w*step_x)+width
            bottom = (h*step_y)+height
            
            # if left-right, top-bottom이 각각 img_w, img_h를 넘지 않을때만 crop을 실행한다
            if (left and right < img_w) and (top and bottom < img_h):
                bbox = (left, top, right, bottom)
                #print(bbox)
        
                # crop
                division_image = pil_img.crop(bbox)
 
                # save 
                fname = "{}.jpg".format("{0:05d}".format(i))
                save_name = save_path + img_file_name + "/" + fname
                #division_image.save(save_name)
            
                i += 1
                division_list.append(division_image)
                fname_list.append(fname)
    
    return division_list, w, h, fname_list

In [4]:
# jpg 또는 png 이미지 파일만 가져오기
img_file_list = [img_file for img_file in os.listdir(img_path_list) if img_file.endswith('.jpg') or img_file.endswith('.png')]

img_list = []

for img_file_name in img_file_list:
    img_path = "/".join([img_path_list, img_file_name])
    pil_img = Image.open(img_path)
    
    division_list, range_w, range_h, fname_list = division_image(img_file_name, pil_img, step_x, step_y, width, height)

    
for img_file_name in img_file_list:
    for fname in fname_list:
    
        ## image → list 전환
        img_file_name = img_file_name.replace(".jpg", "").replace(".png", "")
        img_path = save_path + img_file_name + "/" + fname
        
        org_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        org_img = cv2.resize(org_img, (height,width), interpolation=cv2.INTER_CUBIC)
    
        # 0~1 사이의 값으로 변경하기 위해 '255'로 나눔
        scaled_img = org_img / 255   
        reshaped_img = scaled_img.reshape([height,width,1])
    
        img_list.append(reshaped_img)
    
## list → numpy 변환
img_list = np.array(img_list)
print(img_list.shape)
    
## npy 파일 저장
np.save("/src/hyebin/preprocessing/npy/division_all_img_50", img_list)

print("*** Saved ***")

(206208, 50, 50, 1)
*** Saved ***
